In [ ]:
import openai
import os
import csv
import librosa
import numpy as np

def transcribe_audio(api_key, audio_file):
    client = openai.OpenAI(api_key=api_key)

    with open(audio_file, "rb") as file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=file,
            response_format="verbose_json"
        )

    return response

def compute_harmonic_ratio(audio_file):
    """Compute how much of the voice is concentrated around harmonics."""
    y, sr = librosa.load(audio_file, sr=None)
    harmonic, _ = librosa.effects.hpss(y)
    harmonic_energy = np.sum(harmonic**2)
    total_energy = np.sum(y**2)
    harmonic_ratio = harmonic_energy / total_energy if total_energy > 0 else 0
    return harmonic_ratio

def save_to_csv(results, output_file="/content/output_folder/output.csv"):
    """Save harmonic ratios to a CSV file."""
    os.makedirs("/content/output_folder", exist_ok=True)

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Harmonic Ratio"])
        writer.writerows(results)

def process_multiple_files(api_key, audio_files, output_file):
    results = []

    for audio_file in audio_files:
        print(f"Processing {audio_file}...")
        transcribe_audio(api_key, audio_file)  # Using Whisper API
        harmonic_ratio = compute_harmonic_ratio(audio_file)
        results.append([os.path.basename(audio_file), harmonic_ratio])

    save_to_csv(results, output_file)
    print(f"All data saved to {output_file}")

if __name__ == "__main__":
    API_KEY = ""  # Replace with your OpenAI API key
    AUDIO_FILES = [
        "/content/ABB_segment_2 (1).mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3"
    ]  # Replace with your list of audio files
    OUTPUT_FILE = "/content/output_folder/output_harmonization.csv"

    process_multiple_files(API_KEY, AUDIO_FILES, OUTPUT_FILE)

Processing /content/ABB_segment_2 (1).mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}